In [1]:
import mxnet as mx
from mxnet import gluon,nd,autograd,init
from mxnet.gluon import nn

C:\Users\lenovo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
net = nn.Sequential()
net.add(nn.Dense(256, activation='relu'))
net.add(nn.Dense(10))
net.initialize(ctx=mx.gpu()) # 使⽤默认初始化⽅式。
x = nd.random.uniform(shape=(2, 20),ctx = mx.gpu())
y = net(x) # 前向计算。

In [3]:
y


[[ 0.05329533  0.03159706 -0.01283145 -0.06495545 -0.00498366  0.00298265
   0.08536996  0.00734059 -0.01193844  0.04980217]
 [ 0.06983113  0.02511816  0.01950873 -0.11230142 -0.05404425  0.05385707
   0.02761711 -0.03491186 -0.0081345   0.04493136]]
<NDArray 2x10 @gpu(0)>

## 对于使用Sequential 类构造的神经网络，我们可以通过方括号 [] 来访问网络的任⼀层
## 可以通过 Block 类的 params 属性来访问该层包含的所有参数。

In [4]:
net[0].params

dense0_ (
  Parameter dense0_weight (shape=(256, 20), dtype=float32)
  Parameter dense0_bias (shape=(256,), dtype=float32)
)

In [5]:
net[0].weight.data()


[[ 0.0068339   0.01299825  0.0301265  ... -0.03182812 -0.01631819
  -0.00312688]
 [ 0.0408415   0.04370362  0.00404529 ... -0.05035089  0.0518017
   0.05181222]
 [ 0.06700657 -0.00369488  0.0418822  ... -0.05517294 -0.01194733
  -0.00369594]
 ...
 [ 0.00010955  0.0439323  -0.04911506 ...  0.06975312  0.0449558
  -0.03283203]
 [ 0.04106557  0.05671307 -0.00066976 ...  0.06387014 -0.01292654
   0.00974177]
 [ 0.00297424 -0.0281784  -0.06881659 ... -0.04047417  0.00457048
   0.05696651]]
<NDArray 256x20 @gpu(0)>

In [7]:
net[0].weight.grad()


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
<NDArray 256x20 @gpu(0)>

In [8]:
net.collect_params()

sequential0_ (
  Parameter dense0_weight (shape=(256, 20), dtype=float32)
  Parameter dense0_bias (shape=(256,), dtype=float32)
  Parameter dense1_weight (shape=(10, 256), dtype=float32)
  Parameter dense1_bias (shape=(10,), dtype=float32)
)

In [10]:
#通过正则表达式来获取想要的参数
net.collect_params('.*weight')

sequential0_ (
  Parameter dense0_weight (shape=(256, 20), dtype=float32)
  Parameter dense1_weight (shape=(10, 256), dtype=float32)
)

In [13]:
# ⾮⾸次对模型初始化需要指定 force_reinit 为真。
net.initialize(init=init.Normal(sigma=0.01), force_reinit=True,ctx= mx.gpu())
net[0].weight.data()[0]


[-0.00097133  0.004969    0.00223329  0.00345049  0.01382178 -0.00126281
 -0.02136273 -0.00931591  0.00663047 -0.02017072 -0.01615018 -0.00351595
 -0.00350487 -0.02343885  0.00492284 -0.00832888 -0.0006916  -0.0025537
 -0.00765413  0.00368862]
<NDArray 20 @gpu(0)>

In [18]:
net[0].weight.initialize(init = init.Xavier(),force_reinit=True,ctx=mx.gpu())
net[0].weight.data()


[[ 0.08052795 -0.09453681  0.13527533 ...  0.1155358  -0.02954686
   0.08284207]
 [-0.03528723 -0.08506401 -0.10385555 ... -0.11896712 -0.09759908
   0.14299902]
 [ 0.10603461  0.11694542 -0.13018094 ... -0.0048047  -0.06745408
  -0.01230875]
 ...
 [-0.04861339  0.00849974 -0.06494468 ... -0.03571267  0.04746498
  -0.04032935]
 [-0.13697642 -0.11955108 -0.05871581 ...  0.14606604 -0.01874223
  -0.14503561]
 [-0.06400687 -0.13611342  0.02678049 ... -0.0997941  -0.05755959
   0.07874617]]
<NDArray 256x20 @gpu(0)>

In [19]:
net(x)


[[-0.02486046 -0.00543583 -0.01613953  0.02447063 -0.01559238  0.00911386
   0.00849711  0.01031539 -0.02922858 -0.0161538 ]
 [-0.0168057   0.00166961 -0.04107725 -0.00287132  0.00539324 -0.00119632
   0.02079319  0.02405829 -0.03300514 -0.03673564]]
<NDArray 2x10 @gpu(0)>

## 可以创建一个带有模型参数的自定义层
## 比如说自己实现一个带有 权重 和 偏差 的全连接层

In [21]:
params = gluon.ParameterDict()

In [22]:
params.get('weight',shape=(2,3))

Parameter weight (shape=(2, 3), dtype=<class 'numpy.float32'>)

In [42]:
class MyDense(nn.Block):
    #units为输出单元个数，in_units为输入单元个数
    def __init__(self,units,in_units,**kargs):
        super(MyDense,self).__init__(**kargs)
        self.weight = self.params.get('weight',shape=(in_units,units))
        self.bias = self.params.get('bias',shape=(units,))
    def forward(self,x):
        linear = nd.dot(x.reshape(-1,self.weight.data().shape[0]),self.weight.data())+self.bias.data()
        return nd.relu(linear)

In [43]:
mydense = MyDense(units = 3,in_units=5)

In [44]:
mydense.params

mydense3_ (
  Parameter mydense3_weight (shape=(5, 3), dtype=<class 'numpy.float32'>)
  Parameter mydense3_bias (shape=(3,), dtype=<class 'numpy.float32'>)
)

In [45]:
mydense.initialize(ctx = mx.gpu())

In [46]:
mydense(x)


[[0.02915337 0.00448669 0.        ]
 [0.08164059 0.02985291 0.00102015]
 [0.07324705 0.         0.04513365]
 [0.04066653 0.         0.        ]
 [0.05637019 0.         0.        ]
 [0.07285969 0.         0.023755  ]
 [0.05971614 0.00312614 0.        ]
 [0.03030048 0.0029671  0.        ]]
<NDArray 8x3 @gpu(0)>

## 保存模型参数

In [47]:
mydense.save_parameters('test_params')

In [56]:
net2 = MyDense(units = 3,in_units=5)

In [57]:
net2.load_parameters('test_params',ctx=mx.gpu())

In [58]:
with mx.Context(mx.gpu()):
    net2(x)

In [59]:
  net2(x)


[[0.02915337 0.00448669 0.        ]
 [0.08164059 0.02985291 0.00102015]
 [0.07324705 0.         0.04513365]
 [0.04066653 0.         0.        ]
 [0.05637019 0.         0.        ]
 [0.07285969 0.         0.023755  ]
 [0.05971614 0.00312614 0.        ]
 [0.03030048 0.0029671  0.        ]]
<NDArray 8x3 @gpu(0)>